In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import prompt

template = PromptTemplate.from_template("{region_a}와 {region_b}의 거리는 어떻게 되나요?")

prompt = template.format(region_a="서울", region_b="부산")

# temperature 높을 수록 창의적임 0 ~ 1
chat = ChatOpenAI(temperature=0.1)
chat.predict(prompt)


'서울과 부산은 직선거리로 약 325km 정도 떨어져 있습니다. 그러나 실제로는 도로나 기차를 이용하여 이동할 경우 이 거리는 조금 더 길어질 수 있습니다. 일반적으로 자동차로 이동하는 경우 약 4~5시간 정도 소요되며, KTX를 이용하는 경우에는 약 2시간 30분 정도 소요됩니다.'

In [4]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# SystemMessage: ChatModel의 페르소나, 역할지시
# AIMessage: ChatModel의 답변
# HumanMessage: 사용자의 질문, 요구사항
message = [
    SystemMessage(content="난 지리 전문가이고, {language}로 답해줘요."),
    AIMessage(content="안녕하세요, {name}. 한국어로 답변해드릴게요."),
    HumanMessage(content="{region_a}에서 {region_b}까지 차로가는 거리는 어떻게 돼나요?")
]

chat.predict_messages(message)

AIMessage(content='서울에서 대구까지는 대략 300km 정도의 거리에요. 차로 이동하면 약 4~5시간 정도 소요될 것으로 예상되어요. 다양한 요인에 따라 소요 시간이 달라질 수 있으니 참고해주세요.')

In [ ]:
# ChatPromptTemplate: ChatModel의 페르소나, 역할지시, 답변, 사용자의 질문, 요구사항을 한번에 정의

template = ChatPromptTemplate.from_messages([
    ("system", "난 지리 전문가이고, {language}로 답해줘요."),
    ("ai", "안녕하세요, {name}. 한국어로 답변해드릴게요."),
    ("human", "{region_a}에서 {region_b}까지 차로가는 거리는 어떻게 돼나요?")
])

message = template.format_messages(region_a="서울", region_b="부산", name="김지리", language="한국어")

chat.predict_messages(message)

AIMessage(content='서울에서 부산까지는 대략 400~450km 정도의 거리에요. 고속도로를 이용하면 약 4~5시간 정도 소요되는데, 교통 상황에 따라 달라질 수 있어요. 출발 시간과 경로에 따라 다를 수 있으니 참고해주세요.')

In [21]:
from langchain.schema import BaseOutputParser

class CommaOUtputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        mapObj = map(str.strip, items)
        return list(mapObj)
    
p = CommaOUtputParser()

p.parse("   Hello, how,  are,  you?  ")

['Hello', 'how', 'are', 'you?']

In [34]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 대한 답을 배열형으로 답변을 만들어 내고 comma로 구분해줍니다. 답변의 갯수는 최대 {max_items}개 입니다."),
    ("human", "{question}"),
])

message = template.format_messages(max_items=3, question="행성들이 뭐가 있어?")

result = chat.predict_messages(message)

p = CommaOUtputParser()

p.parse(result.content)

['지구', '화성', '목성', '금성', '토성', '해왕성', '우라누스', '넵튠', '수성', '명왕성']

In [33]:
chain = template | chat | CommaOUtputParser()

chain.invoke({"max_items": 3, "question": "행성들이 뭐가 있어?"})

['1. 수성\n2. 금성\n3. 지구']